### Imports

In [1]:
import pandas as pd
import unidecode
import urllib.parse
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from dotenv import load_dotenv
import os

### Leitura dos csv

In [2]:
extrato = pd.read_csv('../b_extrato_jan.CSV', encoding='latin_1', sep=';', index_col= False, header=1)
alunos = pd.read_csv('../Alunos e valor contribuicao.csv', sep=';', )

# #csvs = os.listdir('..')
# list_csv = []
# for i in csvs:
#     if i.endswith('.CSV'):
#         list_csv.append(i)
# list_csv

In [3]:
# config pandas vizualização de linhas
pd.set_option('display.max_rows', 200)

### limpeza e Tratamento

#### Renomear colunas

In [4]:
extrato = extrato.rename(columns = {'Data' : 'data', 'Lançamento' : 'lancamentos', 'Dcto.' : 'dcto', 'Crédito (R$)' : 'entradas', 'Débito (R$)' : 'saidas'})

#### Drop colunas e linhas

In [5]:
extrato = extrato.drop('Saldo (R$)', axis = 1)
stop = int(extrato[extrato['data'] == 'Total'].index[0]-1)
extrato = extrato.loc[:stop]

#### Tratamento de Float e string

In [6]:
extrato['entradas'] = extrato['entradas'].str.replace('.', '').str.replace(',','.').astype(float)
extrato['saidas'] = extrato['saidas'].str.replace('.', '').str.replace(',','.').astype(float)
extrato['lancamento test'] = extrato['lancamentos'].str.split(':').str[-1].str.lower().str.strip()

C:\Users\rober\AppData\Local\Temp/ipykernel_7692/3034765757.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  extrato['entradas'] = extrato['entradas'].str.replace('.', '').str.replace(',','.').astype(float)
C:\Users\rober\AppData\Local\Temp/ipykernel_7692/3034765757.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  extrato['saidas'] = extrato['saidas'].str.replace('.', '').str.replace(',','.').astype(float)


### Mascaras e filtro

In [7]:
mask = (extrato['lancamentos'] != 'APLIC.INVEST FACIL') & (extrato['lancamentos'] != 'RESGATE INVEST FACIL')
extrato = extrato.loc[mask,:]

#### Separar debito e credito

In [8]:
entradas = extrato.dropna(subset=['entradas'])

In [9]:
saidas = extrato.dropna(subset=['saidas'])

### Pd merge e encontrar pessoas que pagaram a contribução

In [10]:
#CRIAÇÃO DO MERGE
entradas['join'] = 0
alunos['join'] = 0
alunos_uniao = pd.merge(entradas, alunos, left_on= ['join'], right_on=['join'])

C:\Users\rober\AppData\Local\Temp/ipykernel_7692/349170258.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entradas['join'] = 0


In [11]:
maskt = alunos_uniao.apply(lambda x: x['Alunos'].find(x['lancamento test']), axis=1).ge(0)
mask_maior = alunos_uniao.loc[maskt & (alunos_uniao['entradas'] > alunos_uniao['contribuicao']) ,'Alunos']
mask_igual = alunos_uniao.loc[maskt & (alunos_uniao['entradas'] == alunos_uniao['contribuicao']) ,'Alunos']
mask_menor = alunos_uniao.loc[maskt & (alunos_uniao['entradas'] < alunos_uniao['contribuicao']) ,'Alunos']
mask_nulo = alunos_uniao.loc[maskt & (alunos_uniao['entradas']  np.isnan(alunos_uniao['contribuicao'])) ,'Alunos']
#Criação da coluna de confirmação de pagamento
alunos.loc[alunos['Alunos'].isin(mask_maior), 'Pagou?'] = 'MAIOR'
alunos.loc[alunos['Alunos'].isin(mask_igual), 'Pagou?'] = 'OK'
alunos.loc[alunos['Alunos'].isin(mask_menor), 'Pagou?'] = 'MENOR'
alunos.loc[alunos['Alunos'].isin(mask_nulo), 'Pagou?'] = 'nulo'
alunos = alunos.drop(columns= ['join'])

### Fazer um link pro whatsapp com quem não contribuiu


In [12]:
#Criação de links do WhatsApp

def aluno_whats(linha):
    texto_ok = urllib.parse.quote('obrigado')
    texto_nulo = urllib.parse.quote('Oii, mande o comprovante')
    texto_maior = urllib.parse.quote('vi que...')
    texto_menor = urllib.parse.quote('Sobre o que é isso')
    telefone = linha['Telefone']
    if linha['Pagou?'] == "OK":
        link = f'https://api.whatsapp.com/send/?phone={telefone}&text={texto_ok}'
    elif linha['Pagou?'] == 'MAIOR':
        link = f'https://api.whatsapp.com/send/?phone={telefone}&text={texto_maior}'
    elif linha['Pagou?'] == 'MENOR':
        link = f'https://api.whatsapp.com/send/?phone={telefone}&text={texto_menor}'
    else:
        link = f'https://api.whatsapp.com/send/?phone={telefone}&text={texto_nulo}'
    #np.isnan(linha['Pagou?'])
    return link
alunos['link_whats'] = alunos.apply(aluno_whats, axis=1)

## anonimizar dados

## API Google Sheets

In [13]:
#remover nulos e tranformar em lista
# tratamento df alunos
alunos = alunos.fillna('')
planilha_export_controle = alunos.to_numpy().tolist()
# tratamento df entradas
entradas = entradas.drop(columns = ['join'])
entradas = entradas.fillna('')
planilha_export_entradas = entradas.to_numpy().tolist()
# tratamento df saidas
saidas = saidas.fillna('')
planilha_export_saidas = saidas.to_numpy().tolist()

In [ ]:
load_dotenv('../projeto_final_secrets.env')
planilha = os.getenv('linkplanilha')

In [14]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = planilha
SAMPLE_RANGE_NAME = 'TESTE!1:1000'


def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '../client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    main()

In [17]:
#controle de pagamentos

service = build('sheets', 'v4', credentials=Credentials.from_authorized_user_file('token.json', SCOPES))
sheet = service.spreadsheets()
result = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range='TESTE!b3', valueInputOption ='USER_ENTERED',
                           body={'values': planilha_export_controle}).execute()

In [15]:
## entradas

service = build('sheets', 'v4', credentials=Credentials.from_authorized_user_file('token.json', SCOPES))
sheet = service.spreadsheets()
result = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range='entradas!b73', valueInputOption ='USER_ENTERED',
                           body={'values': planilha_export_entradas}).execute()

In [16]:
## saidas

service = build('sheets', 'v4', credentials=Credentials.from_authorized_user_file('token.json', SCOPES))
sheet = service.spreadsheets()
result = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range='saidas!b24', valueInputOption ='USER_ENTERED',
                           body={'values': planilha_export_saidas}).execute()

## WebScraping com Selenium - automatizção de input de dados


In [24]:
caminho = os.getenv('caminho')
site = os.getenv('site')
site2 = os.getenv('site2')
login = os.getenv('login')
senha = os.getenv('password')

In [37]:
def pega_comprovante(df):
    
    nome = df['Alunos']
    if df['Pagou?'] == "OK": 
        
        # passo 1
        chrome_path = caminho
        ser = Service(chrome_path)
        driver = webdriver.Chrome(service=ser)
        #passo 2
        link = site
        driver.get(link)
        time.sleep(1)
        elem = driver.find_element('css selector','body > form > table > tbody > tr:nth-child(1) > td:nth-child(2) > input[type=TEXT]')
        elem.send_keys(login)
        elem = driver.find_element('css selector', 'body > form > table > tbody > tr:nth-child(2) > td:nth-child(2) > input[type=PASSWORD]')
        elem.send_keys(senha)
        elem.send_keys(Keys.RETURN)
            #prevencao #time.sleep(2) # enter na pagina #driver.switch_to.alert.accept()
        #passo 3
        time.sleep(1)
        elem = driver.find_element('css selector', 'body > form > table > tbody > tr:nth-child(15) > td > input:nth-child(5)')
        elem.click()
        # passo 4
        time.sleep(1)
        link2 = site2
        driver.get(link2)
        elem = driver.find_element('css selector', 'body > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(7) > td > a')
        elem.click()
        # passo 5
        driver.switch_to.frame('indice')
        elem = driver.find_element('css selector', 'body > div > table > tbody > tr:nth-child(2) > td > a')
        elem.click()
        # passo 6
        driver.switch_to.default_content()
        driver.switch_to.frame('principal')
        elem = driver.find_element('css selector', 'body > table > tbody > tr:nth-child(11) > td:nth-child(7) > a')
        elem.click()  
        # passo 7
        time.sleep(1)
        elem = driver.find_element('css selector', 'body > table > tbody > tr > td > table:nth-child(1) > tbody > tr > td:nth-child(1) > a')
        elem.click()
        # passo 8
        elem = driver.find_element('css selector', 'body > form > table > tbody > tr:nth-child(1) > td > table:nth-child(1) > tbody > tr > td > input:nth-child(1)')
        elem.click()
        # passo 9 
        elem = driver.find_element('css selector', '#popup > input[type=TEXT]:nth-child(3)')
        elem.send_keys(f'{nome}')
        elem.send_keys(Keys.RETURN)
        # passo 10 
        time.sleep(1)
        elem = driver.find_element('css selector', 'body > form > b:nth-child(8) > table:nth-child(4) > tbody > tr:nth-child(2) > td:nth-child(1) > a')
        elem.click()    
        #passo 11
        time.sleep(1)
        elem = driver.find_element('css selector', 'body > form > table > tbody > tr > td:nth-child(1) > table > tbody > tr:nth-child(3) > td:nth-child(1) > input[type=CHECKBOX]')
        elem.click()
        time.sleep(1)
        elem = driver.find_element('css selector', 'body > form > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(7) > td:nth-child(2)')                                      
        elem.click()
        #passo 12
        time.sleep(2)
        # Com imagem
        elem = driver.find_element('css selector', 'body > form > table > tbody > tr:nth-child(2) > td > table:nth-child(1) > tbody > tr > td > input:nth-child(7)')
        elem.click()
        ## Sem imagem
        ## elem = driver.find_element('css selector', 'body > form > table > tbody > tr:nth-child(2) > td > table:nth-child(1) > tbody > tr > td > input:nth-child(8)')
        ## elem.click()
        driver.switch_to.alert.accept() # enter na pagina 
        time.sleep(1)
        # diga x
        driver.switch_to.window(driver.window_handles[-1]) # trocar aba/janela de controle
        elem = driver.find_element('css selector', 'body > table:nth-child(3)')
        elem.screenshot(f'{nome}_fev.png')
        time.sleep(1)
        driver.close()
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[0])
        driver.close()
    else:
        pass
    

In [43]:
alunos.apply(pega_comprovante, axis=1)